![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Context Engineering with Memory: Building on Your RAG Agent

## From Grounding Problem to Memory Solution

In the previous notebook, you experienced the **grounding problem** - how references break without memory. Now you'll learn to solve this with **sophisticated memory architecture** that enhances your context engineering.

### What You'll Build

Transform your RAG agent with **memory-enhanced context engineering**:

- **🧠 Working Memory** - Session-scoped conversation context
- **📚 Long-term Memory** - Cross-session knowledge and preferences  
- **🔄 Memory Integration** - Seamless working + long-term memory
- **⚡ Agent Memory Server** - Production-ready memory architecture

### Context Engineering Focus

This notebook teaches **memory-enhanced context engineering best practices**:

1. **Memory-Aware Context Assembly** - How memory improves context quality
2. **Reference Resolution** - Using memory to resolve pronouns and references
3. **Personalized Context** - Leveraging long-term memory for personalization
4. **Context Efficiency** - Memory prevents context repetition and bloat
5. **Cross-Session Continuity** - Context that survives across conversations

### Learning Objectives

By the end of this notebook, you will:
1. **Implement** working memory for conversation context
2. **Use** long-term memory for persistent knowledge
3. **Build** memory-enhanced context engineering patterns
4. **Create** agents that remember and learn from interactions
5. **Apply** production-ready memory architecture with Agent Memory Server

## Setup: Agent Memory Server Architecture

We'll use the **Agent Memory Server** - a production-ready memory system that provides:

- **Working Memory** - Session-scoped conversation storage
- **Long-term Memory** - Persistent, searchable knowledge
- **Automatic Extraction** - AI-powered fact extraction from conversations
- **Vector Search** - Semantic search across memories
- **Deduplication** - Prevents redundant memory storage

This is the same architecture used in the `redis_context_course` reference agent.

In [ ]:
# Setup: Import the reference agent components and memory client
import os
import sys
import asyncio
from typing import List, Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

# Load environment
load_dotenv()
sys.path.append('../../reference-agent')

# Import reference agent components
from redis_context_course.models import (
    Course, StudentProfile, DifficultyLevel, 
    CourseFormat, Semester
)
from redis_context_course.course_manager import CourseManager
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import WorkingMemory, MemoryMessage
    MEMORY_SERVER_AVAILABLE = True
    print("✅ Agent Memory Server client available")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("⚠️  Agent Memory Server not available")
    print("📝 Install with: pip install agent-memory-server")
    print("🚀 Start server with: agent-memory-server")

# Verify environment
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found. Please set in .env file.")

print(f"\n🔧 Environment Setup:")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"   AGENT_MEMORY_URL: {os.getenv('AGENT_MEMORY_URL', 'http://localhost:8000')}")
print(f"   Memory Server: {'✓ Available' if MEMORY_SERVER_AVAILABLE else '✗ Not available'}")

## Part 1: Working Memory for Context Engineering

**Working memory** solves the grounding problem by storing conversation context. Let's see how this enhances context engineering.

### Context Engineering Problem Without Memory

Recall from the grounding notebook:
- **Broken references**: "What are its prerequisites?" → Agent doesn't know what "its" refers to
- **Lost context**: Each message is processed in isolation
- **Poor UX**: Users must repeat information

### Context Engineering Solution With Working Memory

Working memory enables **memory-enhanced context engineering**:
- **Reference resolution**: "its" → CS401 (from conversation history)
- **Context continuity**: Each message builds on previous messages
- **Natural conversations**: Users can speak naturally with pronouns and references

In [ ]:
# Initialize Memory Client for working memory
if MEMORY_SERVER_AVAILABLE:
    # Configure memory client
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
        default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    
    print("🧠 Memory Client Initialized")
    print(f"   Base URL: {config.base_url}")
    print(f"   Namespace: {config.default_namespace}")
    print("   Ready for working memory operations")
else:
    print("⚠️  Simulating memory operations (Memory Server not available)")
    memory_client = None

### Working Memory Structure

Working memory contains the essential context for the current conversation:

- **Messages**: The conversation history (user and assistant messages)
- **Session ID**: Identifies this specific conversation
- **User ID**: Identifies the user across sessions
- **Task Data**: Optional task-specific context (current goals, temporary state)

This structure gives the LLM everything it needs to understand the current conversation context.

In [ ]:
# Demonstrate working memory with a conversation that has references
async def demonstrate_working_memory():
    """Show how working memory enables reference resolution in context engineering"""
    
    if not MEMORY_SERVER_AVAILABLE:
        print("📝 This would demonstrate working memory with Agent Memory Server")
        return
    
    # Create a student and session
    student_id = "demo_student_working_memory"
    session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print(f"💬 Starting Conversation with Working Memory")
    print(f"   Student ID: {student_id}")
    print(f"   Session ID: {session_id}")
    print()
    
    # Simulate a conversation with references
    conversation = [
        {"role": "user", "content": "Tell me about RU301 Vector Search"},
        {"role": "assistant", "content": "RU301 Vector Search teaches you to build semantic search with Redis. It covers vector embeddings, similarity search, and practical applications."},
        {"role": "user", "content": "What are its prerequisites?"},  # "its" refers to RU301
        {"role": "assistant", "content": "RU301 requires RU101 (Redis Fundamentals) and RU201 (Redis for Python Developers) as prerequisites."},
        {"role": "user", "content": "Can I take it if I've completed those?"}  # "it" refers to RU301, "those" refers to prerequisites
    ]
    
    # Convert to MemoryMessage format
    memory_messages = [MemoryMessage(**msg) for msg in conversation]
    
    # Create WorkingMemory object
    working_memory = WorkingMemory(
        session_id=session_id,
        user_id=student_id,
        messages=memory_messages,
        memories=[],  # Long-term memories will be added here
        data={}  # Task-specific data
    )
    
    # Store working memory
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )
    
    print("✅ Conversation stored in working memory")
    print(f"📊 Messages stored: {len(conversation)}")
    print()
    
    # Retrieve working memory to show context engineering
    _, retrieved_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        model_name="gpt-4o",
        user_id=student_id
    )
    
    if retrieved_memory:
        print("🎯 Context Engineering with Working Memory:")
        print("   The LLM now has access to full conversation context")
        print("   References can be resolved:")
        print("   • 'its prerequisites' → RU301's prerequisites")
        print("   • 'Can I take it' → Can I take RU301")
        print("   • 'those' → RU101 and RU201")
        print()
        print(f"📋 Retrieved {len(retrieved_memory.messages)} messages from working memory")
        
        return session_id, student_id
    
    return None, None

# Run the demonstration
session_id, student_id = await demonstrate_working_memory()

## Part 2: Long-term Memory for Personalized Context Engineering

**Long-term memory** stores persistent knowledge that enhances context engineering across sessions:

- **Semantic Memory**: Facts and preferences ("Student prefers online courses")
- **Episodic Memory**: Events and experiences ("Student enrolled in CS101 on 2024-09-15")
- **Message Memory**: Important conversation snippets

### Context Engineering Benefits

Long-term memory enables **personalized context engineering**:
- **Preference-aware context**: Include user preferences in context assembly
- **Historical context**: Reference past interactions and decisions
- **Efficient context**: Avoid repeating known information
- **Cross-session continuity**: Context that survives across conversations

In [ ]:
# Demonstrate long-term memory for context engineering
async def demonstrate_long_term_memory():
    """Show how long-term memory enhances context engineering with persistent knowledge"""
    
    if not MEMORY_SERVER_AVAILABLE:
        print("📝 This would demonstrate long-term memory with Agent Memory Server")
        return
    
    print("📚 Long-term Memory for Context Engineering")
    print()
    
    # Store some semantic memories (facts and preferences)
    semantic_memories = [
        "Student prefers online courses over in-person",
        "Student's major is Computer Science",
        "Student wants to specialize in machine learning",
        "Student has completed RU101 and RU201",
        "Student prefers hands-on learning with practical projects"
    ]
    
    user_id = student_id or "demo_student_longterm"
    
    print(f"💾 Storing semantic memories for user: {user_id}")
    
    for memory_text in semantic_memories:
        try:
            await memory_client.create_semantic_memory(
                user_id=user_id,
                text=memory_text
            )
            print(f"   ✅ Stored: {memory_text}")
        except Exception as e:
            print(f"   ⚠️  Could not store: {memory_text} ({e})")
    
    print()
    
    # Search long-term memory to show context engineering benefits
    search_queries = [
        "course preferences",
        "learning style",
        "completed courses",
        "career goals"
    ]
    
    print("🔍 Searching long-term memory for context engineering:")
    
    for query in search_queries:
        try:
            results = await memory_client.search_memories(
                user_id=user_id,
                query=query,
                limit=3
            )
            
            print(f"\n   Query: '{query}'")
            if results:
                for i, result in enumerate(results, 1):
                    print(f"   {i}. {result.text} (score: {result.score:.3f})")
            else:
                print("   No results found")
                
        except Exception as e:
            print(f"   ⚠️  Search failed for '{query}': {e}")
    
    print()
    print("🎯 Context Engineering Impact:")
    print("   • Personalized recommendations based on preferences")
    print("   • Efficient context assembly (no need to re-ask preferences)")
    print("   • Cross-session continuity (remembers across conversations)")
    print("   • Semantic search finds relevant context automatically")

# Run long-term memory demonstration
await demonstrate_long_term_memory()

## Part 3: Memory Integration - Complete Context Engineering

The power of memory-enhanced context engineering comes from **integrating working and long-term memory**.

### Complete Memory Flow for Context Engineering

```
User Query → Agent Processing
     ↓
1. Load Working Memory (conversation context)
     ↓
2. Search Long-term Memory (relevant facts)
     ↓
3. Assemble Enhanced Context:
   • Current conversation (working memory)
   • Relevant preferences (long-term memory)
   • Historical context (long-term memory)
     ↓
4. LLM processes with complete context
     ↓
5. Save response to working memory
     ↓
6. Extract important facts → long-term memory
```

This creates **memory-enhanced context engineering** that provides:
- **Complete context**: Both immediate and historical
- **Personalized context**: Tailored to user preferences
- **Efficient context**: No redundant information
- **Persistent context**: Survives across sessions

### Step 1: Building the Memory-Enhanced RAG Agent Foundation

Let's start by creating the basic structure of our memory-enhanced agent.

In [ ]:
# Build a Memory-Enhanced RAG Agent using reference agent components
class MemoryEnhancedRAGAgent:
    """RAG Agent with sophisticated memory-enhanced context engineering"""
    
    def __init__(self, course_manager: CourseManager, memory_client=None):
        self.course_manager = course_manager
        self.memory_client = memory_client
        self.llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)
    
    async def create_memory_enhanced_context(
        self, 
        student: StudentProfile, 
        query: str, 
        session_id: str,
        courses: List[Course] = None
    ) -> str:
        """🎯 MEMORY-ENHANCED CONTEXT ENGINEERING
        
        This demonstrates advanced context engineering with memory integration.
        
        CONTEXT ENGINEERING ENHANCEMENTS:
        ✅ Working Memory - Current conversation context
        ✅ Long-term Memory - Persistent user knowledge
        ✅ Semantic Search - Relevant memory retrieval
        ✅ Reference Resolution - Pronouns and implicit references
        ✅ Personalization - User-specific context assembly
        """
        
        context_parts = []
        
        # 1. STUDENT PROFILE CONTEXT (Base layer)
        student_context = f"""STUDENT PROFILE:
Name: {student.name}
Email: {student.email}
Major: {student.major}, Year {student.year}
Completed Courses: {', '.join(student.completed_courses) if student.completed_courses else 'None'}
Current Courses: {', '.join(student.current_courses) if student.current_courses else 'None'}
Interests: {', '.join(student.interests)}
Preferred Format: {student.preferred_format.value if student.preferred_format else 'Any'}
Preferred Difficulty: {student.preferred_difficulty.value if student.preferred_difficulty else 'Any'}"""
        
        context_parts.append(student_context)
        
        # 2. LONG-TERM MEMORY CONTEXT (Personalization layer)
        if self.memory_client:
            try:
                # Search for relevant long-term memories
                memory_results = await self.memory_client.search_memories(
                    user_id=student.email,
                    query=query,
                    limit=5
                )
                
                if memory_results:
                    memory_context = "\nRELEVANT MEMORIES:\n"
                    for i, memory in enumerate(memory_results, 1):
                        memory_context += f"{i}. {memory.text}\n"
                    context_parts.append(memory_context)
                    
            except Exception as e:
                print(f"⚠️  Could not retrieve long-term memories: {e}")
        
        # 3. COURSE CONTEXT (RAG layer)
        if courses:
            courses_context = "\nRELEVANT COURSES:\n"
            for i, course in enumerate(courses, 1):
                courses_context += f"""{i}. {course.course_code}: {course.title}
   Description: {course.description}
   Level: {course.difficulty_level.value}
   Format: {course.format.value}
   Credits: {course.credits}
   Prerequisites: {', '.join(course.prerequisites) if course.prerequisites else 'None'}

"""
            context_parts.append(courses_context)
        
        # 4. WORKING MEMORY CONTEXT (Conversation layer)
        if self.memory_client:
            try:
                # Get working memory for conversation context
                _, working_memory = await self.memory_client.get_or_create_working_memory(
                    session_id=session_id,
                    model_name="gpt-3.5-turbo",
                    user_id=student.email
                )
                
                if working_memory and working_memory.messages:
                    conversation_context = "\nCONVERSATION HISTORY:\n"
                    # Show recent messages for reference resolution
                    for msg in working_memory.messages[-6:]:  # Last 6 messages
                        conversation_context += f"{msg.role.title()}: {msg.content}\n"
                    context_parts.append(conversation_context)
                    
            except Exception as e:
                print(f"⚠️  Could not retrieve working memory: {e}")
        
        return "\n".join(context_parts)
    
    async def chat_with_memory(
        self, 
        student: StudentProfile, 
        query: str, 
        session_id: str
    ) -> str:
        """Enhanced chat with complete memory integration"""
        
        # 1. Search for relevant courses
        relevant_courses = await self.course_manager.search_courses(query, limit=3)
        
        # 2. Create memory-enhanced context
        context = await self.create_memory_enhanced_context(
            student, query, session_id, relevant_courses
        )
        
        # 3. Create messages for LLM
        system_message = SystemMessage(content="""You are a helpful academic advisor for Redis University.
Use the provided context to give personalized advice. Pay special attention to:
- Student's learning history and preferences from memories
- Current conversation context for reference resolution
- Course recommendations based on student profile and interests

Be specific, helpful, and reference the student's known preferences and history.""")
        
        human_message = HumanMessage(content=f"""Context:
{context}

Student Question: {query}

Please provide helpful academic advice based on the complete context.""")
        
        # 4. Get LLM response
        response = self.llm.invoke([system_message, human_message])
        
        # 5. Store conversation in working memory
        if self.memory_client:
            await self._update_working_memory(student.email, session_id, query, response.content)
        
        return response.content
    
    async def _update_working_memory(self, user_id: str, session_id: str, user_message: str, assistant_message: str):
        """Update working memory with new conversation turn"""
        try:
            # Get current working memory
            _, working_memory = await self.memory_client.get_or_create_working_memory(
                session_id=session_id,
                model_name="gpt-3.5-turbo",
                user_id=user_id
            )
            
            # Add new messages
            new_messages = [
                MemoryMessage(role="user", content=user_message),
                MemoryMessage(role="assistant", content=assistant_message)
            ]
            
            working_memory.messages.extend(new_messages)
            
            # Save updated working memory
            await self.memory_client.put_working_memory(
                session_id=session_id,
                memory=working_memory,
                user_id=user_id,
                model_name="gpt-3.5-turbo"
            )
            
        except Exception as e:
            print(f"⚠️  Could not update working memory: {e}")

print("🧠 MemoryEnhancedRAGAgent created with sophisticated context engineering!")

## Part 4: Testing Memory-Enhanced Context Engineering

Let's test our memory-enhanced agent to see how it solves the grounding problem and improves context engineering.

In [ ]:
# Test the memory-enhanced agent
async def test_memory_enhanced_context_engineering():
    """Demonstrate how memory solves context engineering challenges"""
    
    # Initialize components
    course_manager = CourseManager()
    agent = MemoryEnhancedRAGAgent(course_manager, memory_client)
    
    # Create test student
    sarah = StudentProfile(
        name='Sarah Chen',
        email='sarah.chen@university.edu',
        major='Computer Science',
        year=3,
        completed_courses=['RU101', 'RU201'],
        current_courses=[],
        interests=['machine learning', 'data science', 'python'],
        preferred_format=CourseFormat.ONLINE,
        preferred_difficulty=DifficultyLevel.INTERMEDIATE,
        max_credits_per_semester=15
    )
    
    # Create session
    test_session_id = f"test_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print("🧪 Testing Memory-Enhanced Context Engineering")
    print(f"   Student: {sarah.name}")
    print(f"   Session: {test_session_id}")
    print()
    
    # Test conversation with references (the grounding problem)
    test_conversation = [
        "Hi! I'm interested in machine learning courses. What do you recommend?",
        "What are the prerequisites for it?",  # "it" should resolve to the recommended ML course
        "I prefer hands-on learning. Does it have practical projects?",  # "it" = same course
        "Perfect! Can I take it next semester?",  # "it" = same course
        "What about the course you mentioned earlier?",  # temporal reference
    ]
    
    for i, query in enumerate(test_conversation, 1):
        print(f"--- Turn {i} ---")
        print(f"👤 Student: {query}")
        
        if MEMORY_SERVER_AVAILABLE:
            try:
                response = await agent.chat_with_memory(sarah, query, test_session_id)
                print(f"🤖 Agent: {response[:200]}..." if len(response) > 200 else f"🤖 Agent: {response}")
            except Exception as e:
                print(f"⚠️  Error: {e}")
        else:
            print("🤖 Agent: [Would respond with memory-enhanced context]")
        
        print()
    
    print("✅ Context Engineering Success:")
    print("   • References resolved using working memory")
    print("   • Personalized responses using long-term memory")
    print("   • Natural conversation flow maintained")
    print("   • No need for users to repeat information")

# Run the test
await test_memory_enhanced_context_engineering()

## Key Takeaways: Memory-Enhanced Context Engineering

### 🎯 **Context Engineering Principles with Memory**

#### **1. Reference Resolution**
- **Working Memory** enables pronoun resolution ("it" → specific course)
- **Conversation History** provides context for temporal references ("you mentioned")
- **Natural Language** patterns work without explicit clarification

#### **2. Personalized Context Assembly**
- **Long-term Memory** provides user preferences and history
- **Semantic Search** finds relevant memories automatically
- **Context Efficiency** avoids repeating known information

#### **3. Cross-Session Continuity**
- **Persistent Knowledge** survives across conversations
- **Learning Accumulation** builds better understanding over time
- **Context Evolution** improves with each interaction

#### **4. Production-Ready Architecture**
- **Agent Memory Server** provides scalable memory management
- **Automatic Extraction** learns from conversations
- **Vector Search** enables semantic memory retrieval
- **Deduplication** prevents redundant memory storage

### 🚀 **Memory-Enhanced Context Engineering Best Practices**

1. **Layer Your Context**:
   - Base: Student profile
   - Personalization: Long-term memories
   - Domain: Relevant courses/content
   - Conversation: Working memory

2. **Enable Reference Resolution**:
   - Store conversation history in working memory
   - Provide recent messages for pronoun resolution
   - Use temporal context for "you mentioned" references

3. **Leverage Semantic Search**:
   - Search long-term memory with user queries
   - Include relevant memories in context
   - Let the system find connections automatically

4. **Optimize Context Efficiency**:
   - Avoid repeating information stored in memory
   - Use memory to reduce context bloat
   - Focus context on new and relevant information

### 🎓 **Next Steps**

You've now mastered **memory-enhanced context engineering**! In Section 4, you'll learn:

- **Tool Selection** - Semantic routing to specialized tools
- **Multi-Tool Coordination** - Memory-aware tool orchestration
- **Advanced Agent Patterns** - Building sophisticated AI assistants

**Your RAG agent now has the memory foundation for advanced AI capabilities!**